# Import Libraries

This needs to be executed once in the beginning when you start the kernel on which the notebook operates.

In [ ]:
import numpy as np

#for interactive plots
%matplotlib notebook 
#When exporting to pdf rplace the "notebook" with "inline" and replot every plot.
#This replaces the interactive plots with regular ones that can be converted with LaTeX.
import matplotlib.pyplot as plt

from PIL import Image

# Load the Data

Please specify the **entire file path and the file name**. Every time you want to analyze a new image, you have to load it here and then execute every step.

In [ ]:
file = r"Sim_3_multi_Ref.tif" #The file has to be in the same folder as this script.
your_folder = r'\\' #The folder where you want to save your data in. Make sure that the folder exists.
prefix = '' #This string will be added in front of your saved image. Choose it according to your current simulation.

image = Image.open(file)
image = image.convert('L')# convert image to grayscale
image = np.asarray(image.getdata(),dtype=np.float64).reshape((image.size[1],image.size[0]))/255

In [ ]:
fig, ax = plt.subplots()
ax = plt.imshow(image, cmap = 'gray')

# Do a Fourier Transform

As a first step, do a Fourier transform of the loaded image and take it's absolute squared. This is what a detector would record.

In [ ]:
hologram = np.abs(np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(image))))**2/np.sqrt(image.size)

## Plot the image

Plot the Fourier trnsform as is and its logarithmic plot.

You can adjust the scale of the image by specifying the mininal (*vmin*) and maximal (*max*) displayed value. You can do this with choosing the percentiles inthe first line but you can also type in any number you like.

In [ ]:
MinV, MaxV = np.percentile(hologram,(2,98))

fig, axs = plt.subplots(1,2)

ax1 = axs[0].imshow(hologram, cmap = 'gray', vmin = MinV, vmax = MaxV)
fig.colorbar(ax1, ax=axs[0], fraction=0.046, pad=0.04)
axs[0].set_title('Fourier trans. (abs squared)')

ax2 = axs[1].imshow(np.log10(hologram), cmap = 'gray', vmin=-2)
fig.colorbar(ax2, ax=axs[1], fraction=0.046, pad=0.04)
axs[1].set_title('Fourier trans. (log abs squared)')

fig.tight_layout()

#save image and data
plt.savefig(your_folder + prefix + 'holo.png')
np.save(your_folder + prefix + 'holo', hologram) #saves as numpy array. can be opened in another notebook with np.load('filepath')

# Reconstruct

Now the artificial hologram is reconstructed. This is the only step that you will execute with the experimental data.

In [ ]:
recon = np.abs(np.fft.ifftshift(np.fft.ifft2(np.fft.fftshift(hologram)))) 

## Plot the image

As before...

In [ ]:
MinV, MaxV = np.percentile(recon,(3,97))

fig, ax = plt.subplots()

ax = plt.imshow(recon, cmap = 'gray', vmin = MinV, vmax = MaxV)
plt.colorbar()
plt.title('Reconstruction (abs)')

fig.tight_layout()

#save image and data
plt.savefig(your_folder + prefix + 'recon.png')
np.save(your_folder + prefix + 'recon', recon) #saves as numpy array. can be opened in another notebook with np.load('filepath')

## Lineout

Do reasonable intensity lineouts to determine the resolution using the 90%-10% criterion.

You can determine the pixelpositions by using the cursor over your image. You can also select an area of the image to enlarge it using the butten left of the floppy disc.

In [ ]:
def lineout(image, startX, endX, startY, endY, file_prefix):
    lineout = np.sum(image[startY:endY, startX:endX], 0)
    
    #plot the lineout
    fig, ax = plt.subplots()
    ax = plt.plot(lineout)
    plt.title("Lineout for resultion")
    plt.xlabel("Pixel")
    plt.ylabel("Intensity [a.u.]")
    
    #save the lineout
    np.savetxt(your_folder + file_prefix + 'lineout.txt', lineout)
    return

In [ ]:
lineout(image=recon, startX = 380, endX = 490, startY = 398, endY = 399, file_prefix=prefix)

If you want to load an existing txt-file of a lineout, you can do this here

In [ ]:
line = np.fromfile(your_folder + 'Sim_3_lineout3.txt', dtype=np.float, sep='\n')

In [ ]:
fig, ax = plt.subplots()
ax.plot(line)

# High Pass Filter

A binary mask will be multiplied to the hologram.

You can set the radius (in pixels) of this mask in the first cell.

In [ ]:
radius = 50

dim_x, dim_y = hologram.shape

xx, yy = np.meshgrid(np.arange(1, hologram.shape[0]+1,1),
                     np.arange(1, hologram.shape[1]+1,1), sparse=True)
#create the mask by setting everything in the radius 0, else 1
mask_HP = np.where((xx-dim_x/2)**2+(yy-dim_y/2)**2<radius**2,0,1)

#muliply mask and holo
holo_HP = mask*hologram

In [ ]:
MinV, MaxV = np.percentile(holo_HP,(2,98))

fig, axs = plt.subplots(1,2)

ax1 = axs[0].imshow(holo_HP, cmap = 'gray', vmin = MinV, vmax = MaxV)
fig.colorbar(ax1, ax=axs[0], fraction=0.046, pad=0.04)
axs[0].set_title('High Pass Hologram')

ax2 = axs[1].imshow(mask_HP, cmap = 'gray')
fig.colorbar(ax2, ax=axs[1], fraction=0.046, pad=0.04)
axs[1].set_title('High Pass Mask')

fig.tight_layout()

#save image and data
plt.savefig(your_folder + 'holo_HP.png')
np.save(your_folder + 'holo_HP', holo_HP) #saves as numpy array. can be opened in another notebook with np.load('filepath')

## Reconstruct

In [ ]:
recon_HP = np.abs(np.fft.ifftshift(np.fft.fft2(np.fft.ifftshift(holo_HP))))

In [ ]:
MinV, MaxV = np.percentile(recon_HP,(3, 97))

fig, ax = plt.subplots()

ax = plt.imshow(recon_HP, cmap = 'gray', vmin = MinV, vmax = MaxV)
plt.colorbar()
plt.title('Reconstruction High Pass (abs)')

fig.tight_layout()

#save image and data
plt.savefig(your_folder + 'recon_HP.png')
np.save(your_folder + 'recon_HP', recon_HP) #saves as numpy array. can be opened in another notebook with np.load('filepath')

## Lineout

In [ ]:
lineout(image=recon_HP, startX = 170, endX = 230, startY = 508, endY = 509, file_suffix='_HP')

# Low Pass Filter

A binary mask will be multiplied to the hologram.

You can set the radius (in pixels) of this mask in the first cell

In [ ]:
radius = 100

dim_x, dim_y = hologram.shape

xx, yy = np.meshgrid(np.arange(1, hologram.shape[0]+1, 1),
                     np.arange(1, hologram.shape[1]+1, 1), sparse=True)
#create the mask by setting everything in the radius 1, else 0
mask_LP = np.where((xx-dim_x/2)**2+(yy-dim_y/2)**2<radius**2, 1, 0)

#muliply mask and holo
holo_LP = mask_LP * hologram

In [ ]:
MinV, MaxV = np.percentile(holo_LP,(2,98))

fig, axs = plt.subplots(1,2)

ax1 = axs[0].imshow(holo_LP, cmap = 'gray', vmin = MinV, vmax = MaxV)
fig.colorbar(ax1, ax=axs[0], fraction=0.046, pad=0.04)
axs[0].set_title('Low Pass Hologram')

ax2 = axs[1].imshow(mask_LP, cmap = 'gray')
fig.colorbar(ax2, ax=axs[1], fraction=0.046, pad=0.04)
axs[1].set_title('Low Pass Mask')

fig.tight_layout()

#save image and data
plt.savefig(your_folder + 'holo_HP.png')
np.save(your_folder + 'holo_HP', holo_LP) #saves as numpy array. can be opened in another notebook with np.load('filepath')

## Reconstruct

In [ ]:
recon_LP = np.abs(np.fft.ifftshift(np.fft.ifft2(np.fft.fftshift(holo_LP))))

In [ ]:
MinV, MaxV = np.percentile(recon_LP,(2.5, 97.5))

fig, ax = plt.subplots()

ax = plt.imshow(recon_LP, cmap = 'gray', vmin = MinV, vmax = MaxV)
plt.colorbar()
plt.title('Reconstruction Low Pass (abs)')

fig.tight_layout()

#save image and data
plt.savefig(your_folder + 'recon_LP.png')
np.save(your_folder + 'recon_LP', recon_LP) #saves as numpy array. can be opened in another notebook with np.load('filepath')

## Lineout

In [ ]:
lineout(image=recon_LP, startX = 170, endX = 230, startY = 508, endY = 509, file_suffix='_LP')